In [46]:
import random
import math

In [47]:
# Define coordinates of cities (you can change or expand this)
cities = {
    0: (0, 0),
    1: (1, 5),
    2: (5, 2),
    3: (6, 6),
    4: (8, 3)
}

In [48]:
num_ants = 10
num_iterations = 100
alpha = 1.0      # pheromone influence
beta = 5.0       # distance influence
evaporation = 0.5
Q = 100

In [49]:
def distance(a, b):
    return math.dist(cities[a], cities[b])

distances = {
    (i, j): distance(i, j) for i in cities for j in cities if i != j
}

# Initialize pheromones
pheromones = {
    (i, j): 1.0 for i in cities for j in cities if i != j
}

In [50]:
def choose_next_city(ant, visited):
    current = ant[-1]
    probabilities = []
    total = 0

    for city in cities:
        if city not in visited:
            pher = pheromones[(current, city)] ** alpha
            heuristic = (1 / distances[(current, city)]) ** beta
            prob = pher * heuristic
            probabilities.append((city, prob))
            total += prob

    r = random.uniform(0, total)
    cumulative = 0
    for city, prob in probabilities:
        cumulative += prob
        if r <= cumulative:
            return city
    return probabilities[-1][0]

In [51]:
# Construct solution
def construct_solution():
    ant = [random.choice(list(cities.keys()))]
    visited = set(ant)

    while len(visited) < len(cities):
        next_city = choose_next_city(ant, visited)
        ant.append(next_city)
        visited.add(next_city)

    ant.append(ant[0])  # Return to start
    return ant

In [52]:
# Calculate total distance of a tour
def tour_length(tour):
    return sum(distances[(tour[i], tour[i+1])] for i in range(len(tour) - 1))

In [53]:
# ACO main loop
best_tour = None
best_length = float("inf")

for iteration in range(num_iterations):
    all_tours = []
    for _ in range(num_ants):
        tour = construct_solution()
        length = tour_length(tour)
        all_tours.append((tour, length))
        if length < best_length:
            best_tour = tour
            best_length = length

    # Evaporate pheromones
    for edge in pheromones:
        pheromones[edge] *= (1 - evaporation)

    # Update pheromones
    for tour, length in all_tours:
        for i in range(len(tour) - 1):
            a, b = tour[i], tour[i+1]
            pheromones[(a, b)] += Q / length
            pheromones[(b, a)] += Q / length  # symmetric

In [54]:
# Final result
print("\nBest Tour Found:")
print(" -> ".join(str(city) for city in best_tour))
print(f"Total Distance: {best_length:.2f}")


Best Tour Found:
0 -> 2 -> 4 -> 3 -> 1 -> 0
Total Distance: 22.35
